In [2]:
import pandas as pd
import numpy as np

# Show all rows
pd.set_option("display.max_rows", None)

# Show all columns
pd.set_option("display.max_columns", None)

# Make sure wide DataFrames don't wrap
pd.set_option("display.width", None)

# Show the full content of each column (no '...')
pd.set_option("display.max_colwidth", None)

In [ ]:
file_path = r"C:\Users\Arunkumara\Downloads\DRGHistoricalReviewInformation_11182025.txt"

df_hist = pd.read_csv(file_path, sep='|', dtype=str, low_memory=False)

print(df_hist.info())
df_hist.head()

In [ ]:
print(df_hist['InitialDenialType'].value_counts())
df_hist_sub = df_hist[df_hist['InitialDenialType']=='Substantive'].copy()
print(df_hist_sub.shape)
print(df_hist_sub['InitialDeterminationStatus'].value_counts())
df_hist_sub['len_adrg'] = df_hist_sub['ADRG'].astype(str).str.len()
df_hist_sub['len_bdrg'] = df_hist_sub['B_DRG'].astype(str).str.len()
print(df_hist_sub['len_adrg'].value_counts())
df_hist_sub_ms_drg = df_hist_sub[(df_hist_sub['len_adrg']==3)].copy()
print(df_hist_sub_ms_drg.shape)
df_hist_sub_ms_drg['AGE'] = (
    df_hist_sub_ms_drg['AGE']
        .astype(str)
        .str.strip()
        #.replace('', pd.NA)
        .astype(float)
)

max_age = df_hist_sub_ms_drg["AGE"].max()
# drop invalid ages
df_hist_sub_ms_drg = df_hist_sub_ms_drg[df_hist_sub_ms_drg["AGE"].between(0, max_age)]

# convert to nullable int
df_hist_sub_ms_drg['AGE'] = df_hist_sub_ms_drg['AGE'].astype('Int64')
print(df_hist_sub_ms_drg.shape)

df_hist_sub_ms_drg['LOS'] = (
    df_hist_sub_ms_drg['LOS']
        .astype(str)            # convert to string
        .str.strip()            # remove spaces
        #.replace('', pd.NA)     # replace '' with NaN
        .astype('float')        # convert safely
        .astype('Int64')        # convert to integer (nullable)
)

df_hist_sub_ms_drg_age_los = df_hist_sub_ms_drg[df_hist_sub_ms_drg['LOS'] >=0]
print(df_hist_sub_ms_drg_age_los.shape)

print(df_hist_sub_ms_drg_age_los['InitialDeterminationStatus'].value_counts())

df_hist_sub_ms_drg_age_los = df_hist_sub_ms_drg_age_los.dropna(subset=["B_DRG"])
df_hist_sub_ms_drg_age_los = df_hist_sub_ms_drg_age_los.dropna(subset=["B_PRIMDX"])

# Step 1: Normalize DRG and A_PDX
df_hist_sub_ms_drg_age_los['ADRG'] =df_hist_sub_ms_drg_age_los['ADRG'].astype(str).str.strip()
df_hist_sub_ms_drg_age_los['PRIM_DX'] = df_hist_sub_ms_drg_age_los['PRIM_DX'].astype(str).str.strip().str.upper()
df_hist_sub_ms_drg_age_los['LOB'] = df_hist_sub_ms_drg_age_los['LOB'].astype(str).str.strip().str.upper()
df_hist_sub_ms_drg_age_los['PaymentType'] = df_hist_sub_ms_drg_age_los['PaymentType'].str.strip().str.upper()

# Step 2: Ensure numeric fields are clean

df_hist_sub_ms_drg_age_los['IDSavings'] = pd.to_numeric(df_hist_sub_ms_drg_age_los['IDSavings'], errors='coerce').fillna(0)
df_hist_sub_ms_drg_age_los['LOS'] = pd.to_numeric(df_hist_sub_ms_drg_age_los['LOS'], errors='coerce').fillna(0)
df_hist_sub_ms_drg_age_los['AGE'] = pd.to_numeric(df_hist_sub_ms_drg_age_los['AGE'], errors='coerce').fillna(0)
df_hist_sub_ms_drg_age_los['A_MCCCount'] = pd.to_numeric(df_hist_sub_ms_drg_age_los['A_MCCCount'], errors='coerce').fillna(0).astype(int)
df_hist_sub_ms_drg_age_los['A_CCCount'] = pd.to_numeric(df_hist_sub_ms_drg_age_los['A_CCCount'], errors='coerce').fillna(0).astype(int)


InitialDenialType
Substantive    772104
Assumptive     110464
Technical        3524
Name: count, dtype: int64
(772104, 282)
InitialDeterminationStatus
Approved    557495
Denied      214609
Name: count, dtype: int64
len_adrg
3    628717
4    143362
2        24
1         1
Name: count, dtype: int64
(628717, 284)
(628716, 284)
(628714, 284)
InitialDeterminationStatus
Approved    465908
Denied      162806
Name: count, dtype: int64


In [ ]:
df_hist_sub_ms_drg_age_los["AGE_bin"] = np.select(
    [
        (df_hist_sub_ms_drg_age_los["AGE"] >= 0) & (df_hist_sub_ms_drg_age_los["AGE"] <= 1),
        (df_hist_sub_ms_drg_age_los["AGE"] >= 2) & (df_hist_sub_ms_drg_age_los["AGE"] <= 5),
        (df_hist_sub_ms_drg_age_los["AGE"] >= 6) & (df_hist_sub_ms_drg_age_los["AGE"] <= 10),
        (df_hist_sub_ms_drg_age_los["AGE"] >= 11) & (df_hist_sub_ms_drg_age_los["AGE"] <= 80),
        (df_hist_sub_ms_drg_age_los["AGE"] > 80)
    ],
    [
        "0-1",
        "2-5",
        "6-10",
        (
            (11 + ((df_hist_sub_ms_drg_age_los["AGE"] - 11) // 10) * 10).astype(int).astype(str)
            + "-"
            + (20 + ((df_hist_sub_ms_drg_age_los["AGE"] - 11) // 10) * 10).astype(int).astype(str)
        ),
        "81+"
    ],
    default=np.nan
)

bin_edges = [-np.inf, 1, 2, 3, 5, 10, 20, 30, 40, 50, 75, np.inf]
bin_labels = [
    '0-1', '1-2', '2-3', '3-5', '5-10',
    '10-20', '20-30', '30-40', '40-50', '50-75', '75+'
]

# Add LOS_Bin to df_apr
df_hist_sub_ms_drg_age_los['LOS'] = pd.to_numeric(df_hist_sub_ms_drg_age_los['LOS'], errors='coerce')
df_hist_sub_ms_drg_age_los['LOS_Bin'] = pd.cut(
    df_hist_sub_ms_drg_age_los['LOS'],
    bins=bin_edges,
    labels=bin_labels,
    right=True,
    include_lowest=True
)

In [ ]:
df_hist_sub_ms_drg_age_los.shape

(622557, 286)

In [ ]:
df_hum = df_hist_sub_ms_drg_age_los[df_hist_sub_ms_drg_age_los['Client']=='Hum']
df_hum.shape

(63309, 286)

In [ ]:
df1 = df_hum.copy()

adx_cols = [f"A_DX{i}" for i in range(2, 26)]
bdx_cols = [f"B_DX{i}" for i in range(2, 26)]

df1["A_DX_List"] = df1[adx_cols].values.tolist()
df1["B_DX_List"] = df1[bdx_cols].values.tolist()

df1["A_DX_List"] = df1["A_DX_List"].apply(lambda x: tuple(sorted([i for i in x if pd.notna(i)])))
df1["B_DX_List"] = df1["B_DX_List"].apply(lambda x: tuple(sorted([i for i in x if pd.notna(i)])))


df2 = df1.copy()

def clean_sdx_list(lst):
    cleaned = []
    for item in lst:
        item = str(item).upper().strip()               # normalize
        item = item.replace("- MCC", "")               # remove MCC
        item = item.replace("- CC", "")                # remove CC
        item = item.replace("MCC", "")                 # safety
        item = item.replace("CC", "")                  # safety
        cleaned.append(item.strip().replace("-", ""))  # final cleanup
    return tuple(cleaned)  # convert to tuple

df2["A_DX_List_Clean"] = df2["A_DX_List"].apply(clean_sdx_list)
df2["B_DX_List_Clean"] = df2["B_DX_List"].apply(clean_sdx_list)


mccandcclist_df = pd.read_excel(r"C:\Users\Arunkumara\Downloads\MCCCCList (1).xlsx")
mccandcclist_df['ICDCode'] = (
    mccandcclist_df['ICDCode']
    .astype(str)
    .str.strip()
    .str.upper()
)
mccandcclist = dict(zip(mccandcclist_df['ICDCode'], mccandcclist_df['MCCorCC']))

def map_sdx_types(icd_list, lookup_dict):
    cleaned_list = []
    for code in icd_list:
        code_clean = str(code).strip().upper()  # normalize

        tag = lookup_dict.get(code_clean, "")  # MCC / CC / General
        cleaned_list.append(f"{code_clean} - {tag}")

    return tuple(cleaned_list)   # convert to tuple

df2['A_DX_Type_list'] = df2['A_DX_List_Clean'].apply(lambda x: map_sdx_types(x, mccandcclist))
df2['B_DX_Type_list'] = df2['B_DX_List_Clean'].apply(lambda x: map_sdx_types(x, mccandcclist))


def extract_sdx_info(sdx_list):

    # If list is empty → return immediately
    if not sdx_list or len(sdx_list) == 0:
        return "No MCC or CC", []

    # Separate entries by type
    mcc = [x.split(" - ")[0] for x in sdx_list if "- MCC" in x]
    cc  = [x.split(" - ")[0] for x in sdx_list if "- CC" in x]
    other = [x.split(" - ")[0] for x in sdx_list 
             if "- MCC" not in x and "- CC" not in x]

    # Determine SDX_Type and SDX_Set
    if mcc:
        return "MCC", mcc
    elif cc:
        return "CC", cc
    elif other:
        return "No MCC or CC", other
    else:
        return "No MCC or CC", []

    
# Apply the logic
df2[["A_DX_Type", "A_DX_Set"]] = df2["A_DX_Type_list"].apply(lambda x: pd.Series(extract_sdx_info(x)))

df2[["B_DX_Type", "B_DX_Set"]] = df2["B_DX_Type_list"].apply(lambda x: pd.Series(extract_sdx_info(x)))


def clean_dx_code(x):
    if pd.isna(x):
        return ""
    
    # Convert to string and normalize
    x = str(x).upper().strip()
    
    # Remove square brackets if any
    x = x.replace("[", "").replace("]", "")
    
    # Remove MCC / CC suffixes
    x = x.replace("- MCC", "").replace("- CC", "")
    x = x.replace("MCC", "").replace("CC", "")
    
    # Remove leftover hyphens
    x = x.replace("-", "")
    
    return x
df2["B_PRIMDX_Clean"] = df2["B_PRIMDX"].apply(clean_dx_code)

def map_sdx_type(code, lookup_dict):
    code_clean = str(code).strip().upper()     # normalize
    tag = lookup_dict.get(code_clean, "")      # MCC / CC / General
    return f"{code_clean} - {tag}"
df2['B_PRIMDX_Mapped'] = df2['B_PRIMDX_Clean'].apply(lambda x: map_sdx_type(x, mccandcclist))

In [ ]:
df2['A_DX_Type'].value_counts()

A_DX_Type
MCC             45829
CC              15451
No MCC or CC     2029
Name: count, dtype: int64

In [ ]:
df2["A_DX_Set_MCC_CC"] = df2.apply(
    lambda row: () if row["A_DX_Type"] == "No MCC or CC" else row["A_DX_Set"],
    axis=1
)

df2["B_DX_Set_MCC_CC"] = df2.apply(
    lambda row: () if row["B_DX_Type"] == "No MCC or CC" else row["B_DX_Set"],
    axis=1
)


In [ ]:
def normalize_dx(x):
    return x.strip().upper() if isinstance(x, str) else x


def classify_pdx(row):
    prim_dx = normalize_dx(row["PRIM_DX"])
    b_prim_dx = normalize_dx(row["B_PRIMDX_Clean"])

    a_sdx = {normalize_dx(x) for x in row["A_DX_List_Clean"]} if row["A_DX_List_Clean"] else set()
    b_sdx = {normalize_dx(x) for x in row["B_DX_List_Clean"]} if row["B_DX_List_Clean"] else set()

    # 1️⃣ Same
    if prim_dx == b_prim_dx:
        return "Same"

    # 2️⃣ Resequence
    if (prim_dx in b_sdx) and (b_prim_dx in a_sdx):
        return "Resequence"

    # 3️⃣ APDX Present in BSDX
    if prim_dx in b_sdx:
        return "APDX Present in BSDX"
    
    if b_prim_dx in a_sdx:
        return "BPDX Present in ASDX"

    # 4️⃣ Removed
    return "Removed"


df2["PDX_Status"] = df2.apply(classify_pdx, axis=1)


In [ ]:
def normalize_dx(x):
    return x.strip().upper() if isinstance(x, str) else x


def classify_pdx(row):
    prim_dx = normalize_dx(row["PRIM_DX"])
    b_prim_dx = normalize_dx(row["B_PRIMDX_Clean"])

    a_sdx = {normalize_dx(x) for x in row["A_DX_Set_MCC_CC"]} if row["A_DX_Set_MCC_CC"] else set()
    b_sdx = {normalize_dx(x) for x in row["B_DX_Set_MCC_CC"]} if row["B_DX_Set_MCC_CC"] else set()

    # 1️⃣ Same
    if prim_dx == b_prim_dx:
        return "Same"

    # 2️⃣ Resequence
    if (prim_dx in b_sdx) and (b_prim_dx in a_sdx):
        return "Resequence"

    # 3️⃣ APDX Present in BSDX
    if prim_dx in b_sdx:
        return "APDX Present in BSDX"
    
    if b_prim_dx in a_sdx:
        return "BPDX Present in ASDX"

    # 4️⃣ Removed
    return "Removed"


df2["PDX__MCC_CC_Status"] = df2.apply(classify_pdx, axis=1)


In [ ]:
df2['PDX_Status'].value_counts()

NameError: name 'df2' is not defined

In [ ]:
df2['PDX__MCC_CC_Status'].value_counts()

PDX__MCC_CC_Status
Same                    57819
Removed                  2896
BPDX Present in ASDX     2409
APDX Present in BSDX      115
Resequence                 70
Name: count, dtype: int64

In [ ]:
df2["A_PDX_Change_Status"] = np.where(
    df2["PRIM_DX"].str.strip().str.upper() ==
    df2["B_PRIMDX_Clean"].str.strip().str.upper(),
    "No Change in PDX",
    "Change in PDX"
)

print(df2['A_PDX_Change_Status'].value_counts())

A_PDX_Change_Status
No Change in PDX    57819
Change in PDX        5490
Name: count, dtype: int64


In [ ]:
def compare_dx_sets(a, b):
    # Convert to string so list formatting does not matter
    a_str = str(a).strip()
    b_str = str(b).strip()

    if a_str == b_str:
        return "No Change"   # Or np.nan if you want blank
    else:
        return f"Change {a_str} → {b_str}"

df2["PDX_What_Change_Status"] = df2.apply(lambda row: compare_dx_sets(row["PRIM_DX"], row["B_PRIMDX_Clean"]),axis=1)

In [27]:
df_Remain = df2[(df2['PDX_Status']=='Removed') |(df2['PDX_Status']=='BPDX Present in ASDX')| (df2['PDX_Status']=='APDX Present in BSDX')]
df_Remain.shape

(5158, 304)

In [55]:
df_Remain.to_excel("df_Remain.xlsx")

In [28]:
df_Remain.head(1)

,Payor,claimid,Client,Control_ID,Claim_Number,PaymentType,Group_Type,Payments,Tax_ID,Provider_ID,HospitalName,LOS,DISP_Code,ADRG,DischargeYear,DischargeMonth,AGE,A_CCCount,A_MCCCount,B_CCCount,B_MCCCount,LOB,MRREQ,MRREC,PRIM_DX,A_DX2,A_DX3,A_DX4,A_DX5,A_DX6,A_DX7,A_DX8,A_DX9,A_DX10,A_DX11,A_DX12,A_DX13,A_DX14,A_DX15,A_DX16,A_DX17,A_DX18,A_DX19,A_DX20,A_DX21,A_DX22,A_DX23,A_DX24,A_DX25,A_POA,A_POA2,A_POA3,A_POA4,A_POA5,A_POA6,A_POA7,A_POA8,A_POA9,A_POA10,A_POA11,A_POA12,A_POA13,A_POA14,A_POA15,A_POA16,A_POA17,A_POA18,A_POA19,A_POA20,A_POA21,A_POA22,A_POA23,A_POA24,A_POA25,A_PX1,A_PX2,A_PX3,A_PX4,A_PX5,A_PX6,A_PX7,A_PX8,A_PX9,A_PX10,A_PX11,A_PX12,A_PX13,A_PX14,A_PX15,A_PX16,A_PX17,A_PX18,A_PX19,A_PX20,A_PX21,A_PX22,A_PX23,A_PX24,A_PX25,B_DISPCode,B_DenialCode,B_DRG,B_PRIMDX,B_DX2,B_DX3,B_DX4,B_DX5,B_DX6,B_DX7,B_DX8,B_DX9,B_DX10,B_DX11,B_DX12,B_DX13,B_DX14,B_DX15,B_DX16,B_DX17,B_DX18,B_DX19,B_DX20,B_DX21,B_DX22,B_DX23,B_DX24,B_DX25,B_POA,B_POA2,B_POA3,B_POA4,B_POA5,B_POA6,B_POA7,B_POA8,B_POA9,B_POA10,B_POA11,B_POA12,B_POA13,B_POA14,B_POA15,B_POA16,B_POA17,B_POA18,B_POA19,B_POA20,B_POA21,B_POA22,B_POA23,B_POA24,B_POA25,B_PX1,B_PX2,B_PX3,B_PX4,B_PX5,B_PX6,B_PX7,B_PX8,B_PX9,B_PX10,B_PX11,B_PX12,B_PX13,B_PX14,B_PX15,B_PX16,B_PX17,B_PX18,B_PX19,B_PX20,B_PX21,B_PX22,B_PX23,B_PX24,B_PX25,A_PRIMDX_Desc,A_DX2_Desc,A_DX3_Desc,A_DX4_Desc,A_DX5_Desc,A_DX6_Desc,A_DX7_Desc,A_DX8_Desc,A_DX9_Desc,A_DX10_Desc,A_DX11_Desc,A_DX12_Desc,A_DX13_Desc,A_DX14_Desc,A_DX15_Desc,A_DX16_Desc,A_DX17_Desc,A_DX18_Desc,A_DX19_Desc,A_DX20_Desc,A_DX21_Desc,A_DX22_Desc,A_DX23_Desc,A_DX24_Desc,A_DX25_Desc,A_PX1_Desc,A_PX2_Desc,A_PX3_Desc,A_PX4_Desc,A_PX5_Desc,A_PX6_Desc,A_PX7_Desc,A_PX8_Desc,A_PX9_Desc,A_PX10_Desc,A_PX11_Desc,A_PX12_Desc,A_PX13_Desc,A_PX14_Desc,A_PX15_Desc,A_PX16_Desc,A_PX17_Desc,A_PX18_Desc,A_PX19_Desc,A_PX20_Desc,A_PX21_Desc,A_PX22_Desc,A_PX23_Desc,A_PX24_Desc,A_PX25_Desc,B_PRIMDX_Desc,B_DX2_Desc,B_DX3_Desc,B_DX4_Desc,B_DX5_Desc,B_DX6_Desc,B_DX7_Desc,B_DX8_Desc,B_DX9_Desc,B_DX10_Desc,B_DX11_Desc,B_DX12_Desc,B_DX13_Desc,B_DX14_Desc,B_DX15_Desc,B_DX16_Desc,B_DX17_Desc,B_DX18_Desc,B_DX19_Desc,B_DX20_Desc,B_DX21_Desc,B_DX22_Desc,B_DX23_Desc,B_DX24_Desc,B_DX25_Desc,B_PX1_Desc,B_PX2_Desc,B_PX3_Desc,B_PX4_Desc,B_PX5_Desc,B_PX6_Desc,B_PX7_Desc,B_PX8_Desc,B_PX9_Desc,B_PX10_Desc,B_PX11_Desc,B_PX12_Desc,B_PX13_Desc,B_PX14_Desc,B_PX15_Desc,B_PX16_Desc,B_PX17_Desc,B_PX18_Desc,B_PX19_Desc,B_PX20_Desc,B_PX21_Desc,B_PX22_Desc,B_PX23_Desc,B_PX24_Desc,B_PX25_Desc,InitialDeterminationDate,InitialDenialType,InitialDeterminationStatus,IDSavings,AdjustedSavings,len_adrg,len_bdrg,AGE_bin,LOS_Bin,A_DX_List,B_DX_List,A_DX_List_Clean,B_DX_List_Clean,A_DX_Type_list,B_DX_Type_list,A_DX_Type,A_DX_Set,B_DX_Type,B_DX_Set,B_PRIMDX_Clean,B_PRIMDX_Mapped,A_DX_Set_MCC_CC,B_DX_Set_MCC_CC,PDX_Status,PDX__MCC_CC_Status,A_PDX_Change_Status,PDX_What_Change_Status
33873,Strat,834673,Hum,HUPTMR-26211,820240190358713,POSTPAY,01,13057.77,362167864,000076354B,ST ANTHONY MEDICAL CENTER,18,03-TRNSF TO SKILLED NURSING FAC,871,2024,1,75,4,3,4,2,MEP3I,2024-03-14 00:00:00,2024-04-05 00:00:00,A419,G928 - MCC,L89153 - MCC,J690 - MCC,D84821 - CC,N179 - CC,Z6843 - CC,B3789 - CC,J449,M179,K219,I10,G2581,G4733,E6601,E785,I4891,M1120,E890,T380X5A,M4690,Z87891,Z7982,Z7952,R0689,Y,Y,Y,N,Y,N,NaN,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,NaN,NaN,NaN,N,05H933Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,03-TRNSF TO SKILLED NURSING FAC,801C,091,G928 - MCC,L89153 - MCC,J690 - MCC,D84821 - CC,N179 - CC,Z6843 - CC,B3789 - CC,J449,M179,K219,I10,G2581,G4733,E6601,E785,I4891,M1120,E890,T380X5A,M4690,Z87891,Z7982,Z7952,R0689,NaN,Y,Y,N,Y,N,1,N,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,Y,1,1,1,N,NaN,05H933Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Sepsis, unspecified organism",Other toxic encephalopathy,"Pressure ulcer of sacral region, stage 3",Pneumonitis due to inhalation of food and vomit,Immunodeficiency due to drugs,"Acute kidney failure,

In [105]:
import numpy as np

# ---------------------------------------
# 1. Explode A_DX_List_Clean → SDX
# ---------------------------------------
df_exp = (
    df_Remain
    .explode('A_DX_List_Clean')
    .rename(columns={'A_DX_List_Clean': 'SDX'})
)

for col in ['SDX', 'B_PRIMDX_Clean', 'PRIM_DX']:
    df_exp[col] = df_exp[col].astype(str).str.strip().str.upper()


# ---------------------------------------
# 2. Single PRIM_DX aggregation
# ---------------------------------------
single_agg = (
    df_Remain
    .groupby('PRIM_DX')
    .agg(
        Total_Claim_Single_Value=('PRIM_DX', 'size'),
        Approved_Claim_Single_Value=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_Single_Value=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        )
    )
    .reset_index()
)

# ---------------------------------------
# 3. Tuple (PRIM_DX, SDX) aggregation
# ---------------------------------------
tuple_agg = (
    df_exp
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_Tuple=('SDX', 'size'),
        Approved_Claim_Tuple=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_Tuple=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        ),
        Total_Saving=('IDSavings', 'sum')
    )
    .reset_index()
)

# ---------------------------------------
# 4A. SDX == B_PRIMDX_Clean aggregation
# ---------------------------------------
sdx_bpdx_agg = (
    df_exp[df_exp['SDX'] == df_exp['B_PRIMDX_Clean']]
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_SDX_BPDX=('SDX', 'size'),
        Approved_Claim_SDX_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_SDX_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        ),
        Total_Saving_SDX_BPDX=('IDSavings', 'sum'),
        Avg_Saving_SDX_BPDX_Per_Claim=('IDSavings', 'mean')
    )
    .reset_index()
)

# ---------------------------------------
# 4B. SDX != B_PRIMDX_Clean aggregation
# ---------------------------------------
sdx_not_bpdx_agg = (
    df_exp[df_exp['SDX'] != df_exp['B_PRIMDX_Clean']]
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_SDX_Not_BPDX=('SDX', 'size'),
        Approved_Claim_SDX_Not_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_SDX_Not_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        ),
        Total_Saving_SDX_Not_BPDX=('IDSavings', 'sum')
    )
    .reset_index()
)

# ---------------------------------------
# 5. B_PRIMDX_Clean lookup (safe join)
# ---------------------------------------
bpdx_lookup = (
    df_exp[['PRIM_DX', 'SDX', 'B_PRIMDX_Clean']]
    .drop_duplicates()
)

# ---------------------------------------
# 6. Merge all results
# ---------------------------------------
result_df = (
    tuple_agg
    .merge(single_agg, on='PRIM_DX', how='left')
    .merge(sdx_bpdx_agg, on=['PRIM_DX', 'SDX'], how='left')
    .merge(sdx_not_bpdx_agg, on=['PRIM_DX', 'SDX'], how='left')
    .merge(bpdx_lookup, on=['PRIM_DX', 'SDX'], how='left')
    .fillna({
        'Total_Claim_SDX_BPDX': 0,
        'Approved_Claim_SDX_BPDX': 0,
        'Denied_Claim_SDX_BPDX': 0,
        'Total_Saving_SDX_BPDX': 0,
        'Avg_Saving_SDX_BPDX_Per_Claim':0,
        'Total_Claim_SDX_Not_BPDX': 0,
        'Approved_Claim_SDX_Not_BPDX': 0,
        'Denied_Claim_SDX_Not_BPDX': 0,
        'Total_Saving_SDX_Not_BPDX': 0
    })
    .sort_values(by='Total_Claim_Tuple', ascending=False)
    .reset_index(drop=True)
)

# ---------------------------------------
# 7. Flag column (optional but recommended)
# ---------------------------------------
result_df['SDX_vs_BPRIMDX_Flag'] = np.where(
    result_df['SDX'] == result_df['B_PRIMDX_Clean'],
    'SDX = B_PRIMDX',
    'SDX ≠ B_PRIMDX'
)

# ---------------------------------------
# 8. Final shape
# ---------------------------------------
result_df.shape


(63122, 20)

In [107]:
dff = df_exp[(df_exp['SDX']=='N390') & (df_exp['B_PRIMDX_Clean']=='N390') ]
dff.shape

(466, 304)

In [108]:
result_df.head(1)

,PRIM_DX,SDX,Total_Claim_Tuple,Approved_Claim_Tuple,Denied_Claim_Tuple,Total_Saving,Total_Claim_Single_Value,Approved_Claim_Single_Value,Denied_Claim_Single_Value,Total_Claim_SDX_BPDX,Approved_Claim_SDX_BPDX,Denied_Claim_SDX_BPDX,Total_Saving_SDX_BPDX,Avg_Saving_SDX_BPDX_Per_Claim,Total_Claim_SDX_Not_BPDX,Approved_Claim_SDX_Not_BPDX,Denied_Claim_SDX_Not_BPDX,Total_Saving_SDX_Not_BPDX,B_PRIMDX_Clean,SDX_vs_BPRIMDX_Flag
0,A419,N179,786,11,775,3739187.9,2045,29,2016,25.0,1.0,24.0,70915.5,2836.62,761.0,10.0,751.0,3668272.4,N136,SDX ≠ B_PRIMDX


In [109]:
result_df11 = result_df[result_df['B_PRIMDX_Clean']=='N390']
result_df11.shape

(3914, 20)

In [110]:
result_df1 = result_df[result_df['SDX_vs_BPRIMDX_Flag']=='SDX = B_PRIMDX']
result_df1.shape

(1454, 20)

In [111]:
result_df1 = result_df1.sort_values(
    by='Denied_Claim_SDX_BPDX',
    ascending=False
).reset_index(drop=True)


In [112]:
result_df1 = result_df1.rename(columns={
    'Total_Claim_Single_Value': 'Total_Claim_PDX',
    'Approved_Claim_Single_Value': 'Approved_Claim_PDX',
    'Denied_Claim_Single_Value': 'Denied_Claim_PDX',
    'Total_Claim_Tuple': 'Total_Claim_SDX',
    'Approved_Claim_Tuple': 'Approved_Claim_SDX',
    'Denied_Claim_Tuple': 'Denied_Claim_SDX',
    'B_PRIMDX_Clean': 'B_PDX'
})

result_df1.head()


,PRIM_DX,SDX,Total_Claim_SDX,Approved_Claim_SDX,Denied_Claim_SDX,Total_Saving,Total_Claim_PDX,Approved_Claim_PDX,Denied_Claim_PDX,Total_Claim_SDX_BPDX,Approved_Claim_SDX_BPDX,Denied_Claim_SDX_BPDX,Total_Saving_SDX_BPDX,Avg_Saving_SDX_BPDX_Per_Claim,Total_Claim_SDX_Not_BPDX,Approved_Claim_SDX_Not_BPDX,Denied_Claim_SDX_Not_BPDX,Total_Saving_SDX_Not_BPDX,B_PDX,SDX_vs_BPRIMDX_Flag
0,A419,J189,588,7,581,3122896.34,2045,29,2016,382.0,4.0,378.0,2048803.12,5363.358953,206.0,3.0,203.0,1074093.22,J189,SDX = B_PRIMDX
1,J9621,J441,231,3,228,438053.91,434,6,428,208.0,3.0,205.0,397983.93,1913.384279,23.0,0.0,23.0,40069.98,J441,SDX = B_PRIMDX
2,A419,N390,412,3,409,1948635.90,2045,29,2016,204.0,0.0,204.0,985836.71,4832.532892,208.0,3.0,205.0,962799.19,N390,SDX = B_PRIMDX
3,J9601,J441,148,1,147,318110.32,609,2,607,129.0,0.0,129.0,280321.16,2173.032248,19.0,1.0,18.0,37789.16,J441,SDX = B_PRIMDX
4,A419,J690,204,3,201,767438.96,2045,29,2016,99.0,0.0,99.0,269599.45,2723.226768,105.0,3.0,102.0,497839.51,J690,SDX = B_PRIMDX


In [113]:
result_df2 = result_df1[
    [
        'PRIM_DX',
        'Total_Claim_PDX',
        'Approved_Claim_PDX',
        'Denied_Claim_PDX',
        'SDX',
        'Total_Claim_SDX',
        'Approved_Claim_SDX',
        'Denied_Claim_SDX',
        'B_PDX',
        'Total_Claim_SDX_BPDX',
        'Approved_Claim_SDX_BPDX',
        'Denied_Claim_SDX_BPDX',
        'Total_Saving_SDX_BPDX',
        'Avg_Saving_SDX_BPDX_Per_Claim'
        
    ]
]
result_df2.head()

,PRIM_DX,Total_Claim_PDX,Approved_Claim_PDX,Denied_Claim_PDX,SDX,Total_Claim_SDX,Approved_Claim_SDX,Denied_Claim_SDX,B_PDX,Total_Claim_SDX_BPDX,Approved_Claim_SDX_BPDX,Denied_Claim_SDX_BPDX,Total_Saving_SDX_BPDX,Avg_Saving_SDX_BPDX_Per_Claim
0,A419,2045,29,2016,J189,588,7,581,J189,382.0,4.0,378.0,2048803.12,5363.358953
1,J9621,434,6,428,J441,231,3,228,J441,208.0,3.0,205.0,397983.93,1913.384279
2,A419,2045,29,2016,N390,412,3,409,N390,204.0,0.0,204.0,985836.71,4832.532892
3,J9601,609,2,607,J441,148,1,147,J441,129.0,0.0,129.0,280321.16,2173.032248
4,A419,2045,29,2016,J690,204,3,201,J690,99.0,0.0,99.0,269599.45,2723.226768


In [114]:
result_df2.to_excel("s4.xlsx")

# for all hum

In [115]:
df2.shape

(63309, 304)

In [123]:
df2.to_excel("hum.xlsx")

In [116]:
import numpy as np

# ---------------------------------------
# 1. Explode A_DX_List_Clean → SDX
# ---------------------------------------
df_exp = (
    df2
    .explode('A_DX_List_Clean')
    .rename(columns={'A_DX_List_Clean': 'SDX'})
)

for col in ['SDX', 'B_PRIMDX_Clean', 'PRIM_DX']:
    df_exp[col] = df_exp[col].astype(str).str.strip().str.upper()


# ---------------------------------------
# 2. Single PRIM_DX aggregation
# ---------------------------------------
single_agg = (
    df2
    .groupby('PRIM_DX')
    .agg(
        Total_Claim_Single_Value=('PRIM_DX', 'size'),
        Approved_Claim_Single_Value=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_Single_Value=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        )
    )
    .reset_index()
)

# ---------------------------------------
# 3. Tuple (PRIM_DX, SDX) aggregation
# ---------------------------------------
tuple_agg = (
    df_exp
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_Tuple=('SDX', 'size'),
        Approved_Claim_Tuple=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_Tuple=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        ),
        Total_Saving=('IDSavings', 'sum')
    )
    .reset_index()
)

# ---------------------------------------
# 4A. SDX == B_PRIMDX_Clean aggregation
# ---------------------------------------
sdx_bpdx_agg = (
    df_exp[df_exp['SDX'] == df_exp['B_PRIMDX_Clean']]
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_SDX_BPDX=('SDX', 'size'),
        Approved_Claim_SDX_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_SDX_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        ),
        Total_Saving_SDX_BPDX=('IDSavings', 'sum'),
        Avg_Saving_SDX_BPDX_Per_Claim=('IDSavings', 'mean')
    )
    .reset_index()
)

# ---------------------------------------
# 4B. SDX != B_PRIMDX_Clean aggregation
# ---------------------------------------
sdx_not_bpdx_agg = (
    df_exp[df_exp['SDX'] != df_exp['B_PRIMDX_Clean']]
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_SDX_Not_BPDX=('SDX', 'size'),
        Approved_Claim_SDX_Not_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_SDX_Not_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        ),
        Total_Saving_SDX_Not_BPDX=('IDSavings', 'sum')
    )
    .reset_index()
)

# ---------------------------------------
# 5. B_PRIMDX_Clean lookup (safe join)
# ---------------------------------------
bpdx_lookup = (
    df_exp[['PRIM_DX', 'SDX', 'B_PRIMDX_Clean']]
    .drop_duplicates()
)

# ---------------------------------------
# 6. Merge all results
# ---------------------------------------
result_df = (
    tuple_agg
    .merge(single_agg, on='PRIM_DX', how='left')
    .merge(sdx_bpdx_agg, on=['PRIM_DX', 'SDX'], how='left')
    .merge(sdx_not_bpdx_agg, on=['PRIM_DX', 'SDX'], how='left')
    .merge(bpdx_lookup, on=['PRIM_DX', 'SDX'], how='left')
    .fillna({
        'Total_Claim_SDX_BPDX': 0,
        'Approved_Claim_SDX_BPDX': 0,
        'Denied_Claim_SDX_BPDX': 0,
        'Total_Saving_SDX_BPDX': 0,
        'Avg_Saving_SDX_BPDX_Per_Claim':0,
        'Total_Claim_SDX_Not_BPDX': 0,
        'Approved_Claim_SDX_Not_BPDX': 0,
        'Denied_Claim_SDX_Not_BPDX': 0,
        'Total_Saving_SDX_Not_BPDX': 0
    })
    .sort_values(by='Total_Claim_Tuple', ascending=False)
    .reset_index(drop=True)
)

# ---------------------------------------
# 7. Flag column (optional but recommended)
# ---------------------------------------
result_df['SDX_vs_BPRIMDX_Flag'] = np.where(
    result_df['SDX'] == result_df['B_PRIMDX_Clean'],
    'SDX = B_PRIMDX',
    'SDX ≠ B_PRIMDX'
)

# ---------------------------------------
# 8. Final shape
# ---------------------------------------
result_df.shape


(381044, 20)

In [117]:
result_df1 = result_df[result_df['SDX_vs_BPRIMDX_Flag']=='SDX = B_PRIMDX']
result_df1.shape

(1666, 20)

In [118]:
result_df1 = result_df1.sort_values(
    by='Denied_Claim_SDX_BPDX',
    ascending=False
).reset_index(drop=True)


In [119]:
result_df1 = result_df1.rename(columns={
    'Total_Claim_Single_Value': 'Total_Claim_PDX',
    'Approved_Claim_Single_Value': 'Approved_Claim_PDX',
    'Denied_Claim_Single_Value': 'Denied_Claim_PDX',
    'Total_Claim_Tuple': 'Total_Claim_SDX',
    'Approved_Claim_Tuple': 'Approved_Claim_SDX',
    'Denied_Claim_Tuple': 'Denied_Claim_SDX',
    'B_PRIMDX_Clean': 'B_PDX'
})

result_df1.head()


,PRIM_DX,SDX,Total_Claim_SDX,Approved_Claim_SDX,Denied_Claim_SDX,Total_Saving,Total_Claim_PDX,Approved_Claim_PDX,Denied_Claim_PDX,Total_Claim_SDX_BPDX,Approved_Claim_SDX_BPDX,Denied_Claim_SDX_BPDX,Total_Saving_SDX_BPDX,Avg_Saving_SDX_BPDX_Per_Claim,Total_Claim_SDX_Not_BPDX,Approved_Claim_SDX_Not_BPDX,Denied_Claim_SDX_Not_BPDX,Total_Saving_SDX_Not_BPDX,B_PDX,SDX_vs_BPRIMDX_Flag
0,A419,J189,1333,743,590,3194873.36,4413,2354,2059,384.0,4.0,380.0,2058626.09,5361.005443,949.0,739.0,210.0,1136247.27,J189,SDX = B_PRIMDX
1,J9621,J441,1024,785,239,456294.71,1985,1536,449,217.0,3.0,214.0,405313.89,1867.805945,807.0,782.0,25.0,50980.82,J441,SDX = B_PRIMDX
2,A419,N390,897,480,417,2017105.04,4413,2354,2059,205.0,0.0,205.0,991667.65,4837.403171,692.0,480.0,212.0,1025437.39,N390,SDX = B_PRIMDX
3,J9601,J441,395,241,154,323988.28,1712,1088,624,136.0,0.0,136.0,286199.12,2104.405294,259.0,241.0,18.0,37789.16,J441,SDX = B_PRIMDX
4,A419,J690,492,286,206,805135.26,4413,2354,2059,100.0,0.0,100.0,272055.61,2720.556100,392.0,286.0,106.0,533079.65,J690,SDX = B_PRIMDX


In [121]:
result_df2 = result_df1[
    [
        'PRIM_DX',
        'Total_Claim_PDX',
        'Approved_Claim_PDX',
        'Denied_Claim_PDX',
        'SDX',
        'Total_Claim_SDX',
        'Approved_Claim_SDX',
        'Denied_Claim_SDX',
        'B_PDX',
        'Total_Claim_SDX_BPDX',
        'Approved_Claim_SDX_BPDX',
        'Denied_Claim_SDX_BPDX',
       # 'Total_Saving_SDX_BPDX',
       # 'Avg_Saving_SDX_BPDX_Per_Claim'
        
    ]
]
result_df2.head()

,PRIM_DX,Total_Claim_PDX,Approved_Claim_PDX,Denied_Claim_PDX,SDX,Total_Claim_SDX,Approved_Claim_SDX,Denied_Claim_SDX,B_PDX,Total_Claim_SDX_BPDX,Approved_Claim_SDX_BPDX,Denied_Claim_SDX_BPDX
0,A419,4413,2354,2059,J189,1333,743,590,J189,384.0,4.0,380.0
1,J9621,1985,1536,449,J441,1024,785,239,J441,217.0,3.0,214.0
2,A419,4413,2354,2059,N390,897,480,417,N390,205.0,0.0,205.0
3,J9601,1712,1088,624,J441,395,241,154,J441,136.0,0.0,136.0
4,A419,4413,2354,2059,J690,492,286,206,J690,100.0,0.0,100.0


In [122]:
result_df2.to_excel('hum.xlsx')

# prev

In [73]:
# ---------------------------------------
# 1. Explode A_DX_List_Clean → SDX
# ---------------------------------------
df_exp = (
    df_Remain
    .explode('A_DX_List_Clean')
    .rename(columns={'A_DX_List_Clean': 'SDX'})
)

# ---------------------------------------
# 2. Single PRIM_DX aggregation
# ---------------------------------------
single_agg = (
    df_Remain
    .groupby('PRIM_DX')
    .agg(
        Total_Claim_Single_Value=('PRIM_DX', 'size'),
        Approved_Claim_Single_Value=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_Single_Value=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        )
    )
    .reset_index()
)

# ---------------------------------------
# 3. Tuple (PRIM_DX, SDX) aggregation
# ---------------------------------------
tuple_agg = (
    df_exp
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_Tuple=('SDX', 'size'),
        Approved_Claim_Tuple=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_Tuple=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        ),
        Total_Saving=('IDSavings', 'sum')
    )
    .reset_index()
)

# ---------------------------------------
# 4. SDX == B_PRIMDX_Clean aggregation
# ---------------------------------------
sdx_bpdx_agg = (
    df_exp[df_exp['SDX'] == df_exp['B_PRIMDX_Clean']]
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_SDX_BPDX=('SDX', 'size'),
        Approved_Claim_SDX_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() == 'approved').sum()
        ),
        Denied_Claim_SDX_BPDX=(
            'InitialDeterminationStatus',
            lambda x: (x.str.lower() != 'approved').sum()
        ),
        Total_Saving_SDX_BPDX=('IDSavings', 'sum')
    )
    .reset_index()
)

# ---------------------------------------
# 5. Merge all results
# ---------------------------------------
result_df = (
    tuple_agg
    .merge(single_agg, on='PRIM_DX', how='left')
    .merge(sdx_bpdx_agg, on=['PRIM_DX', 'SDX'], how='left')
    .fillna({
        'Total_Claim_SDX_BPDX': 0,
        'Approved_Claim_SDX_BPDX': 0,
        'Denied_Claim_SDX_BPDX': 0,
        'Total_Saving_SDX_BPDX': 0
    })
    .sort_values(by='Total_Claim_SDX_BPDX', ascending=False)
    .reset_index(drop=True)
)

# ---------------------------------------
# 6. Final shape
# ---------------------------------------
result_df.shape


(21121, 13)

In [74]:
result_df.head()

,PRIM_DX,SDX,Total_Claim_Tuple,Approved_Claim_Tuple,Denied_Claim_Tuple,Total_Saving,Total_Claim_Single_Value,Approved_Claim_Single_Value,Denied_Claim_Single_Value,Total_Claim_SDX_BPDX,Approved_Claim_SDX_BPDX,Denied_Claim_SDX_BPDX,Total_Saving_SDX_BPDX
0,N179,E860,169,5,164,157014.67,471,15,456,86.0,4.0,82.0,75617.09
1,J9621,J439,91,0,91,208528.76,434,6,428,47.0,0.0,47.0,116762.51
2,N179,E875,96,3,93,84349.11,471,15,456,37.0,0.0,37.0,29595.00
3,J9601,J439,66,0,66,180157.27,609,2,607,37.0,0.0,37.0,107306.63
4,A419,E1169,80,1,79,479824.89,2045,29,2016,28.0,0.0,28.0,164616.37


In [75]:
bpdx_lookup = (
    df_exp[['PRIM_DX', 'SDX', 'B_PRIMDX_Clean']]
    .drop_duplicates()
)


In [76]:
result_df = (
    result_df
    .merge(
        bpdx_lookup,
        on=['PRIM_DX', 'SDX'],
        how='left'
    )
)


In [78]:
result_df = result_df.rename(columns={
    'Total_Claim_Single_Value': 'Total_Claim_PDX',
    'Approved_Claim_Single_Value': 'Approved_Claim_PDX',
    'Denied_Claim_Single_Value': 'Denied_Claim_PDX',
    'Total_Claim_Tuple': 'Total_Claim_SDX',
    'Approved_Claim_Tuple': 'Approved_Claim_SDX',
    'Denied_Claim_Tuple': 'Denied_Claim_SDX',
    'B_PRIMDX_Clean': 'B_PDX'
})

result_df.head()


,PRIM_DX,SDX,Total_Claim_SDX,Approved_Claim_SDX,Denied_Claim_SDX,Total_Saving,Total_Claim_PDX,Approved_Claim_PDX,Denied_Claim_PDX,Total_Claim_SDX_BPDX,Approved_Claim_SDX_BPDX,Denied_Claim_SDX_BPDX,Total_Saving_SDX_BPDX,B_PDX
0,N179,E860,169,5,164,157014.67,471,15,456,86.0,4.0,82.0,75617.09,E860
1,N179,E860,169,5,164,157014.67,471,15,456,86.0,4.0,82.0,75617.09,E11649
2,N179,E860,169,5,164,157014.67,471,15,456,86.0,4.0,82.0,75617.09,N3001
3,N179,E860,169,5,164,157014.67,471,15,456,86.0,4.0,82.0,75617.09,N3000
4,N179,E860,169,5,164,157014.67,471,15,456,86.0,4.0,82.0,75617.09,E870


In [80]:
result_df = result_df[
    [
        'PRIM_DX',
        'Total_Claim_PDX',
        'Approved_Claim_PDX',
        'Denied_Claim_PDX',
        'SDX',
        'Total_Claim_SDX',
        'Approved_Claim_SDX',
        'Denied_Claim_SDX',
        'B_PDX',
        'Total_Claim_SDX_BPDX',
        'Approved_Claim_SDX_BPDX',
        'Denied_Claim_SDX_BPDX'
        
    ]
]
result_df.head()

,PRIM_DX,Total_Claim_PDX,Approved_Claim_PDX,Denied_Claim_PDX,SDX,Total_Claim_SDX,Approved_Claim_SDX,Denied_Claim_SDX,B_PDX,Total_Claim_SDX_BPDX,Approved_Claim_SDX_BPDX,Denied_Claim_SDX_BPDX
0,N179,471,15,456,E860,169,5,164,E860,86.0,4.0,82.0
1,N179,471,15,456,E860,169,5,164,E11649,86.0,4.0,82.0
2,N179,471,15,456,E860,169,5,164,N3001,86.0,4.0,82.0
3,N179,471,15,456,E860,169,5,164,N3000,86.0,4.0,82.0
4,N179,471,15,456,E860,169,5,164,E870,86.0,4.0,82.0


In [33]:
df_exp = (
    df_Remain
    .explode('A_DX_List_Clean')
    .rename(columns={'A_DX_List_Clean': 'SDX'})
)


In [34]:
df_exp['In_BPRIMDX'] = df_exp['SDX'] == df_exp['B_PRIMDX_Clean']


In [35]:
single_agg = (
    df_Remain
    .groupby('PRIM_DX')
    .agg(
        Total_Claim_Single_Value=('PRIM_DX', 'size'),
        Approved_Claim_Single_Value=('InitialDeterminationStatus', lambda x: (x.str.lower() == 'approved').sum()),
        Denied_Claim_Single_Value=('InitialDeterminationStatus', lambda x: (x.str.lower() != 'approved').sum())
    )
    .reset_index()
)


In [ ]:
tuple_agg = (
    df_exp
    .groupby(['PRIM_DX', 'SDX'])
    .agg(
        Total_Claim_Tuple=('SDX', 'size'),
        Approved_Claim_Tuple=('InitialDeterminationStatus', lambda x: (x.str.lower() == 'approved').sum()),
        Denied_Claim_Tuple=('InitialDeterminationStatus', lambda x: (x.str.lower() != 'approved').sum()),
        Total_Saving=('IDSavings', 'sum'),
     
    )
    .reset_index()
)


In [37]:
tuple_agg['Not_In_BPRIMDX_Value'] = tuple_agg['SDX'].where(
    ~tuple_agg['In_BPRIMDX'], ""
)


In [67]:
result_df = (
    tuple_agg
    .merge(single_agg, on='PRIM_DX', how='left')
   # .drop(columns='In_BPRIMDX')
    .sort_values(by='Denied_Claim_Tuple', ascending=False)
    .reset_index(drop=True)
)
result_df.shape

(21121, 9)

,PRIM_DX,SDX,Total_Claim_SDX,Approved_Claim_SDX,Denied_Claim_SDX,Total_Saving,Total_Claim_PDX,Approved_Claim_PDX,Denied_Claim_PDX
0,A419,N179,786,11,775,3739187.90,2045,29,2016
1,A419,E785,766,8,758,3597331.76,2045,29,2016
2,A419,I10,689,11,678,3131786.89,2045,29,2016
3,A419,J189,588,7,581,3122896.34,2045,29,2016
4,A419,Z20822,553,2,551,2398158.42,2045,29,2016


In [70]:
result_df.head()

,PRIM_DX,Total_Claim_PDX,Approved_Claim_PDX,Denied_Claim_PDX,SDX,Total_Claim_SDX,Approved_Claim_SDX,Denied_Claim_SDX
0,A419,2045,29,2016,N179,786,11,775
1,A419,2045,29,2016,E785,766,8,758
2,A419,2045,29,2016,I10,689,11,678
3,A419,2045,29,2016,J189,588,7,581
4,A419,2045,29,2016,Z20822,553,2,551


In [54]:
result_df.to_excel("S2.xlsx")

#  prev

In [28]:
df_Same = df2[(df2['PDX_Status']=='Same')]
df_Same.shape

(57819, 304)

In [29]:
df_Same['InitialDeterminationStatus'].value_counts()

InitialDeterminationStatus
Approved    55272
Denied       2547
Name: count, dtype: int64

In [19]:
df_Resequence = df2[(df2['PDX_Status']=='Resequence')]
df_Resequence.shape

(332, 304)

In [20]:
df_Resequence.to_excel("Hum_df_Resequence.xlsx")

In [31]:
df_Resequence['InitialDeterminationStatus'].value_counts()

InitialDeterminationStatus
Denied      326
Approved      6
Name: count, dtype: int64

In [32]:
df_Resequence.head(1)

,Payor,claimid,Client,Control_ID,Claim_Number,PaymentType,Group_Type,Payments,Tax_ID,Provider_ID,HospitalName,LOS,DISP_Code,ADRG,DischargeYear,DischargeMonth,AGE,A_CCCount,A_MCCCount,B_CCCount,B_MCCCount,LOB,MRREQ,MRREC,PRIM_DX,A_DX2,A_DX3,A_DX4,A_DX5,A_DX6,A_DX7,A_DX8,A_DX9,A_DX10,A_DX11,A_DX12,A_DX13,A_DX14,A_DX15,A_DX16,A_DX17,A_DX18,A_DX19,A_DX20,A_DX21,A_DX22,A_DX23,A_DX24,A_DX25,A_POA,A_POA2,A_POA3,A_POA4,A_POA5,A_POA6,A_POA7,A_POA8,A_POA9,A_POA10,A_POA11,A_POA12,A_POA13,A_POA14,A_POA15,A_POA16,A_POA17,A_POA18,A_POA19,A_POA20,A_POA21,A_POA22,A_POA23,A_POA24,A_POA25,A_PX1,A_PX2,A_PX3,A_PX4,A_PX5,A_PX6,A_PX7,A_PX8,A_PX9,A_PX10,A_PX11,A_PX12,A_PX13,A_PX14,A_PX15,A_PX16,A_PX17,A_PX18,A_PX19,A_PX20,A_PX21,A_PX22,A_PX23,A_PX24,A_PX25,B_DISPCode,B_DenialCode,B_DRG,B_PRIMDX,B_DX2,B_DX3,B_DX4,B_DX5,B_DX6,B_DX7,B_DX8,B_DX9,B_DX10,B_DX11,B_DX12,B_DX13,B_DX14,B_DX15,B_DX16,B_DX17,B_DX18,B_DX19,B_DX20,B_DX21,B_DX22,B_DX23,B_DX24,B_DX25,B_POA,B_POA2,B_POA3,B_POA4,B_POA5,B_POA6,B_POA7,B_POA8,B_POA9,B_POA10,B_POA11,B_POA12,B_POA13,B_POA14,B_POA15,B_POA16,B_POA17,B_POA18,B_POA19,B_POA20,B_POA21,B_POA22,B_POA23,B_POA24,B_POA25,B_PX1,B_PX2,B_PX3,B_PX4,B_PX5,B_PX6,B_PX7,B_PX8,B_PX9,B_PX10,B_PX11,B_PX12,B_PX13,B_PX14,B_PX15,B_PX16,B_PX17,B_PX18,B_PX19,B_PX20,B_PX21,B_PX22,B_PX23,B_PX24,B_PX25,A_PRIMDX_Desc,A_DX2_Desc,A_DX3_Desc,A_DX4_Desc,A_DX5_Desc,A_DX6_Desc,A_DX7_Desc,A_DX8_Desc,A_DX9_Desc,A_DX10_Desc,A_DX11_Desc,A_DX12_Desc,A_DX13_Desc,A_DX14_Desc,A_DX15_Desc,A_DX16_Desc,A_DX17_Desc,A_DX18_Desc,A_DX19_Desc,A_DX20_Desc,A_DX21_Desc,A_DX22_Desc,A_DX23_Desc,A_DX24_Desc,A_DX25_Desc,A_PX1_Desc,A_PX2_Desc,A_PX3_Desc,A_PX4_Desc,A_PX5_Desc,A_PX6_Desc,A_PX7_Desc,A_PX8_Desc,A_PX9_Desc,A_PX10_Desc,A_PX11_Desc,A_PX12_Desc,A_PX13_Desc,A_PX14_Desc,A_PX15_Desc,A_PX16_Desc,A_PX17_Desc,A_PX18_Desc,A_PX19_Desc,A_PX20_Desc,A_PX21_Desc,A_PX22_Desc,A_PX23_Desc,A_PX24_Desc,A_PX25_Desc,B_PRIMDX_Desc,B_DX2_Desc,B_DX3_Desc,B_DX4_Desc,B_DX5_Desc,B_DX6_Desc,B_DX7_Desc,B_DX8_Desc,B_DX9_Desc,B_DX10_Desc,B_DX11_Desc,B_DX12_Desc,B_DX13_Desc,B_DX14_Desc,B_DX15_Desc,B_DX16_Desc,B_DX17_Desc,B_DX18_Desc,B_DX19_Desc,B_DX20_Desc,B_DX21_Desc,B_DX22_Desc,B_DX23_Desc,B_DX24_Desc,B_DX25_Desc,B_PX1_Desc,B_PX2_Desc,B_PX3_Desc,B_PX4_Desc,B_PX5_Desc,B_PX6_Desc,B_PX7_Desc,B_PX8_Desc,B_PX9_Desc,B_PX10_Desc,B_PX11_Desc,B_PX12_Desc,B_PX13_Desc,B_PX14_Desc,B_PX15_Desc,B_PX16_Desc,B_PX17_Desc,B_PX18_Desc,B_PX19_Desc,B_PX20_Desc,B_PX21_Desc,B_PX22_Desc,B_PX23_Desc,B_PX24_Desc,B_PX25_Desc,InitialDeterminationDate,InitialDenialType,InitialDeterminationStatus,IDSavings,AdjustedSavings,len_adrg,len_bdrg,AGE_bin,LOS_Bin,A_DX_List,B_DX_List,A_DX_List_Clean,B_DX_List_Clean,A_DX_Type_list,B_DX_Type_list,A_DX_Type,A_DX_Set,B_DX_Type,B_DX_Set,B_PRIMDX_Clean,B_PRIMDX_Mapped,PDX_Status,PDX__MCC_CC_Status,A_PDX_Change_Status,PDX_What_Change_Status,A_DX_Set_MCC_CC,B_DX_Set_MCC_CC
107951,Strat,833970,Hum,HUPTMR-25419,820240020466298,POSTPAY,01,12688.77,223601678,223601678,TRINITAS REGIONAL MEDICAL CENT,4,01-ROUTINE/HOME,190,2023,12,60,1,2,1,2,MEP3I,2024-03-14 00:00:00,2024-04-05 00:00:00,J441,J9601 - MCC,J9602 - MCC,J45901 - CC,I10,E785,Z87891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,Y,Y,Y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5A09357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01-ROUTINE/HOME,801D,202,J45901 - CC,J9601 - MCC,J9602 - MCC,J441 - CC,I10,E785,Z87891,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Y,Y,Y,Y,Y,Y,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5A09357,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Chronic obstructive pulmonary disease with (acute) exacerbation,Acute respiratory failure with hypoxia,Acute respiratory failure with hypercapnia,Unspecified asthma with (acute) exacerbation,Essential (primary) hypertension,"Hyperlipidemia, unspecified",Personal h

In [33]:
from collections import defaultdict

# -----------------------------
# 5. Initialize stats dictionary
# -----------------------------
# Key: (DRG, PRIM_DX, SDX_Set, LOS_Bin)
stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

# -----------------------------
# 6. Loop through df_apr
# -----------------------------
for idx, row in df_Resequence.iterrows():        
    apdx = str(row['PRIM_DX']).strip().upper()
    bpdx = str(row['B_PRIMDX_Clean']).strip().upper()
    pdx_chg = row['PDX_What_Change_Status']
    pdx_status = row['PDX_Status']
    audit_result = row['IDSavings']
   # chap_set = row['CHAPTER_SET']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'



    # Group key includes LOS_Bin
    group_key = (apdx,bpdx,pdx_status,)

    # Update stats
    stats[group_key]['Total_Claims'] += 1

    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result > 0:
            stats[group_key]['Total_Savings'] += float(audit_result)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    apdx,bpdx,pdx_status = key
    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)

    denial_percent = round(denied / total_claims * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0

    final_data.append({
       # 'DRG': drg,
        'A_PRIM_DX': apdx,
        'B_PRIM_DX': bpdx,
        #'PDX_Change_Status': pdx_chg,
        'PDX_Status': pdx_status,
        'Total_Claims': total_claims,
        'Approved': approved,
        'Denied': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_Per_Claim': avg_saving_per_claim
    })

# -----------------------------
# 8. Export to Excel
# -----------------------------
output_df = pd.DataFrame(final_data)
output_df = output_df.sort_values(
    by=['Total_Claims'],
    ascending=[False]
)
output_df.head()


# output_file = "DRG_PDX_SDXSet_LOSBin_Simplified_Analysis.xlsx"
# output_df.to_excel(output_file, index=False)

#print(f"\n✅ Simplified LOS-Bin analysis completed for {len(output_df)} groups.")
# print(f"📊 Results saved to '{output_file}'")

,A_PRIM_DX,B_PRIM_DX,PDX_Status,Total_Claims,Approved,Denied,Denial_Percent,Total_Savings,Avg_Saving_Per_Claim
72,J9621,J441,Resequence,9,0,9,100.0,7329.96,814.44
29,J9621,J439,Resequence,8,0,8,100.0,6149.68,768.71
6,I110,J441,Resequence,7,0,7,100.0,16766.37,2395.20
19,J9601,J441,Resequence,7,0,7,100.0,5877.96,839.71
62,I674,I161,Resequence,5,0,5,100.0,8933.88,1786.78


In [35]:
output_df.to_excel("df_PDX_resequence1.xlsx")

In [17]:
df_Removed = df2[(df2['PDX_Status']=='Removed')]
df_Removed.shape

(218, 304)

In [18]:
df_Removed.to_excel("Hum_APDX_Remove.xlsx")

In [37]:
df_Removed['InitialDeterminationStatus'].value_counts()

InitialDeterminationStatus
Denied      211
Approved      7
Name: count, dtype: int64

In [40]:
from collections import defaultdict

# -----------------------------
# 5. Initialize stats dictionary
# -----------------------------
# Key: (DRG, PRIM_DX, SDX_Set, LOS_Bin)
stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

# -----------------------------
# 6. Loop through df_apr
# -----------------------------
for idx, row in df_Removed.iterrows():        
    apdx = str(row['PRIM_DX']).strip().upper()
    bpdx = str(row['B_PRIMDX_Clean']).strip().upper()
    pdx_chg = row['PDX_What_Change_Status']
    pdx_status = row['PDX_Status']
    audit_result = row['IDSavings']
   # chap_set = row['CHAPTER_SET']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'



    # Group key includes LOS_Bin
    group_key = (apdx,bpdx)

    # Update stats
    stats[group_key]['Total_Claims'] += 1

    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result > 0:
            stats[group_key]['Total_Savings'] += float(audit_result)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    apdx,bpdx = key
    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)

    denial_percent = round(denied / total_claims * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0

    final_data.append({
       # 'DRG': drg,
        'A_PRIM_DX': apdx,
        'B_PRIM_DX': bpdx,
       # 'PDX_Change_Status': pdx_chg,
        'PDX_Status': pdx_status,
        'Total_Claims': total_claims,
        'Approved': approved,
        'Denied': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_Per_Claim': avg_saving_per_claim
    })

# -----------------------------
# 8. Export to Excel
# -----------------------------
output_df = pd.DataFrame(final_data)
output_df = output_df.sort_values(
    by=['Total_Claims'],
    ascending=[False]
)
output_df.head()


# output_file = "DRG_PDX_SDXSet_LOSBin_Simplified_Analysis.xlsx"
# output_df.to_excel(output_file, index=False)

#print(f"\n✅ Simplified LOS-Bin analysis completed for {len(output_df)} groups.")
# print(f"📊 Results saved to '{output_file}'")

,A_PRIM_DX,B_PRIM_DX,PDX_Status,Total_Claims,Approved,Denied,Denial_Percent,Total_Savings,Avg_Saving_Per_Claim
8,J9601,R0902,Removed,19,0,19,100.00,41714.04,2195.48
12,A419,J189,Removed,17,0,17,100.00,74725.44,4395.61
7,E222,E871,Removed,15,2,13,86.67,24480.40,1632.03
5,G9341,R4182,Removed,9,0,9,100.00,15039.25,1671.03
10,J690,J189,Removed,9,0,9,100.00,14908.15,1656.46


In [41]:
output_df.to_excel("df_PDX_remove1.xlsx")

In [16]:
df_BPDX_ASDX = df2[(df2['PDX_Status']=='BPDX Present in ASDX')]
df_BPDX_ASDX.shape

(4928, 304)

In [21]:
df_BPDX_ASDX.to_excel("hum_df_BPDX_ASDX.xlsx")

In [52]:
df_BPDX_ASDX['InitialDeterminationStatus'].value_counts()

InitialDeterminationStatus
Denied      4846
Approved      82
Name: count, dtype: int64

In [44]:
from collections import defaultdict

# -----------------------------
# 5. Initialize stats dictionary
# -----------------------------
# Key: (DRG, PRIM_DX, SDX_Set, LOS_Bin)
stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

# -----------------------------
# 6. Loop through df_apr
# -----------------------------
for idx, row in df_BPDX_ASDX.iterrows():        
    apdx = str(row['PRIM_DX']).strip().upper()
    bpdx = str(row['B_PRIMDX_Clean']).strip().upper()
    pdx_chg = row['PDX_What_Change_Status']
    pdx_status = row['PDX_Status']
    audit_result = row['IDSavings']
   # chap_set = row['CHAPTER_SET']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'



    # Group key includes LOS_Bin
    group_key = (apdx,bpdx)

    # Update stats
    stats[group_key]['Total_Claims'] += 1

    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result > 0:
            stats[group_key]['Total_Savings'] += float(audit_result)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    apdx,bpdx = key
    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)

    denial_percent = round(denied / total_claims * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0

    final_data.append({
       # 'DRG': drg,
        'A_PRIM_DX': apdx,
        'B_PRIM_DX': bpdx,
        #'PDX_Change_Status': pdx_chg,
        'PDX_Status': pdx_status,
        'Total_Claims': total_claims,
        'Approved': approved,
        'Denied': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_Per_Claim': avg_saving_per_claim
    })

# -----------------------------
# 8. Export to Excel
# -----------------------------
output_df = pd.DataFrame(final_data)
output_df = output_df.sort_values(
    by=['Total_Claims'],
    ascending=[False]
)
output_df.head()


# output_file = "DRG_PDX_SDXSet_LOSBin_Simplified_Analysis.xlsx"
# output_df.to_excel(output_file, index=False)

#print(f"\n✅ Simplified LOS-Bin analysis completed for {len(output_df)} groups.")
# print(f"📊 Results saved to '{output_file}'")

,A_PRIM_DX,B_PRIM_DX,PDX_Status,Total_Claims,Approved,Denied,Denial_Percent,Total_Savings,Avg_Saving_Per_Claim
3,A419,J189,BPDX Present in ASDX,382,4,378,98.95,2048803.12,5363.36
17,J9621,J441,BPDX Present in ASDX,208,4,204,98.08,397983.93,1913.38
20,A419,N390,BPDX Present in ASDX,204,0,204,100.00,985836.71,4832.53
9,J9601,J441,BPDX Present in ASDX,129,1,128,99.22,280321.16,2173.03
31,A419,J690,BPDX Present in ASDX,99,0,99,100.00,269599.45,2723.23


In [46]:
output_df.to_excel("df_BPDX_ASDX1.xlsx")

In [47]:
df_APDX_BSDX = df2[(df2['PDX_Status']=='APDX Present in BSDX')]
df_APDX_BSDX.shape

(12, 304)

In [48]:
df_APDX_BSDX.to_excel("Hum_Data_APDX in BSDX.xlsx")

In [49]:
df_APDX_BSDX['InitialDeterminationStatus'].value_counts()

InitialDeterminationStatus
Denied    12
Name: count, dtype: int64

In [50]:
from collections import defaultdict

# -----------------------------
# 5. Initialize stats dictionary
# -----------------------------
# Key: (DRG, PRIM_DX, SDX_Set, LOS_Bin)
stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

# -----------------------------
# 6. Loop through df_apr
# -----------------------------
for idx, row in df_APDX_BSDX.iterrows():        
    apdx = str(row['PRIM_DX']).strip().upper()
    bpdx = str(row['B_PRIMDX_Clean']).strip().upper()
    pdx_chg = row['PDX_What_Change_Status']
    pdx_status = row['PDX_Status']
    audit_result = row['IDSavings']
   # chap_set = row['CHAPTER_SET']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'



    # Group key includes LOS_Bin
    group_key = (apdx,bpdx)

    # Update stats
    stats[group_key]['Total_Claims'] += 1

    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result > 0:
            stats[group_key]['Total_Savings'] += float(audit_result)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    apdx,bpdx = key
    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)

    denial_percent = round(denied / total_claims * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0

    final_data.append({
       # 'DRG': drg,
        'A_PRIM_DX': apdx,
        'B_PRIM_DX': bpdx,
        #'PDX_Change_Status': pdx_chg,
        'PDX_Status': pdx_status,
        'Total_Claims': total_claims,
        'Approved': approved,
        'Denied': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_Per_Claim': avg_saving_per_claim
    })

# -----------------------------
# 8. Export to Excel
# -----------------------------
output_df = pd.DataFrame(final_data)
output_df = output_df.sort_values(
    by=['Total_Claims'],
    ascending=[False]
)
output_df.head()


# output_file = "DRG_PDX_SDXSet_LOSBin_Simplified_Analysis.xlsx"
# output_df.to_excel(output_file, index=False)

#print(f"\n✅ Simplified LOS-Bin analysis completed for {len(output_df)} groups.")
# print(f"📊 Results saved to '{output_file}'")

,A_PRIM_DX,B_PRIM_DX,PDX_Status,Total_Claims,Approved,Denied,Denial_Percent,Total_Savings,Avg_Saving_Per_Claim
0,N184,I130,APDX Present in BSDX,1,0,1,100.0,293.25,293.25
1,D61810,I110,APDX Present in BSDX,1,0,1,100.0,11959.84,11959.84
2,N139,N401,APDX Present in BSDX,1,0,1,100.0,2259.81,2259.81
3,I350,R55,APDX Present in BSDX,1,0,1,100.0,4645.63,4645.63
4,T8744,E1169,APDX Present in BSDX,1,0,1,100.0,887.52,887.52


In [51]:
output_df.to_excel("df_APDX_BSDX1.xlsx")

# PREV

In [ ]:
df2["A_PDX_Change_Status"] = np.where(
    df2["PRIM_DX"].str.strip().str.upper() ==
    df2["B_PRIMDX_Clean"].str.strip().str.upper(),
    "No Change in PDX",
    "Resequence"
)

print(df2['A_PDX_Change_Status'].value_counts())

In [ ]:
def compare_dx_sets(a, b):
    # Convert to string so list formatting does not matter
    a_str = str(a).strip()
    b_str = str(b).strip()

    if a_str == b_str:
        return "No Change"   # Or np.nan if you want blank
    else:
        return f"Change {a_str} → {b_str}"

df2["PDX_What_Change_Status"] = df2.apply(lambda row: compare_dx_sets(row["PRIM_DX"], row["B_PRIMDX_Clean"]),axis=1)

In [ ]:
import numpy as np

def tag_prim_dx(row):
    prim = str(row["PRIM_DX"]).strip().upper()
    b_prim = str(row["B_PRIMDX_Clean"]).strip().upper()

    # Normalize SDX tuple
    sdx = row["B_DX_List_Clean"]
    if isinstance(sdx, (list, tuple)):
        sdx_norm = {str(x).strip().upper() for x in sdx}
    else:
        sdx_norm = set()

    # Rule priority
    if prim == b_prim:
        return "Present in B_PDX"
    elif prim in sdx_norm:
        return "Present in B_SDX"
    else:
        return "A_PDX_Removed"


df2["A_PDX_Presence_Tag"] = df2.apply(tag_prim_dx, axis=1)


In [ ]:
import numpy as np

def tag_prim_dx(row):
    prim = str(row["B_PRIMDX_Clean"]).strip().upper()
    b_prim = str(row["PRIM_DX"]).strip().upper()

    # Normalize SDX tuple
    sdx = row["A_DX_List_Clean"]
    if isinstance(sdx, (list, tuple)):
        sdx_norm = {str(x).strip().upper() for x in sdx}
    else:
        sdx_norm = set()

    # Rule priority
    if prim == b_prim:
        return "Present in A_PDX"
    elif prim in sdx_norm:
        return "Present in A_SDX"
    else:
        return "B_PDX_Added"


df2["B_PDX_Presence_Tag"] = df2.apply(tag_prim_dx, axis=1)


In [ ]:
df_PDX_resequence = df2[df2['A_PDX_Change_Status']=='Resequence']
df_PDX_resequence.shape

In [ ]:
df_PDX_resequence_1 = df_PDX_resequence[(df_PDX_resequence['A_PDX_Presence_Tag']=='A_PDX_Removed')]
df_PDX_resequence_1.shape

In [ ]:
df_PDX_resequence_1.head()

In [ ]:
df_PDX_resequence_1['B_PDX_Presence_Tag'].value_counts()

In [ ]:
from collections import defaultdict

# -----------------------------
# 5. Initialize stats dictionary
# -----------------------------
# Key: (DRG, PRIM_DX, SDX_Set, LOS_Bin)
stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

# -----------------------------
# 6. Loop through df_apr
# -----------------------------
for idx, row in df_PDX_resequence.iterrows():
    # Skip if LOS_Bin is missing
    #if pd.isna(row['PDX_Change_Status']):
       # continue
        
    drg = row['ADRG']
    pdx = str(row['PRIM_DX']).strip().upper()
    apdx_status = row['A_PDX_Presence_Tag']
    bpdx_status = row['B_PDX_Presence_Tag']
    audit_result = row['IDSavings']
   # chap_set = row['CHAPTER_SET']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'



    # Group key includes LOS_Bin
    group_key = (pdx,apdx_status,bpdx_status)

    # Update stats
    stats[group_key]['Total_Claims'] += 1

    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result > 0:
            stats[group_key]['Total_Savings'] += float(audit_result)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    pdx,apdx_status,bpdx_status = key
    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)

    denial_percent = round(denied / total_claims * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0

    final_data.append({
       # 'DRG': drg,
        'PRIM_DX': pdx,
        'A_PDX_Status': apdx_status,
        'B_PDX_Status': bpdx_status,
        'Total_Claims': total_claims,
        'Approved': approved,
        'Denied': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_Per_Claim': avg_saving_per_claim
    })

# -----------------------------
# 8. Export to Excel
# -----------------------------
output_df = pd.DataFrame(final_data)
output_df = output_df.sort_values(
    by=['Total_Claims'],
    ascending=[False]
)
output_df.head()


# output_file = "DRG_PDX_SDXSet_LOSBin_Simplified_Analysis.xlsx"
# output_df.to_excel(output_file, index=False)

#print(f"\n✅ Simplified LOS-Bin analysis completed for {len(output_df)} groups.")
# print(f"📊 Results saved to '{output_file}'")

In [ ]:
from collections import defaultdict

# -----------------------------
# 5. Initialize stats dictionary
# -----------------------------
# Key: (DRG, PRIM_DX, SDX_Set, LOS_Bin)
stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

# -----------------------------
# 6. Loop through df_apr
# -----------------------------
for idx, row in df_PDX_resequence.iterrows():
    # Skip if LOS_Bin is missing
    #if pd.isna(row['PDX_Change_Status']):
     #   continue
        
    drg = row['ADRG']
    pdx = str(row['PRIM_DX']).strip().upper()
    pdx_chg = row['PDX_What_Change_Status']
    apdx_status = row['A_PDX_Presence_Tag']
    bpdx_status = row['B_PDX_Presence_Tag']
   # age_bin = row['AGE_bin']
    audit_result = row['IDSavings']
   # chap_set = row['CHAPTER_SET']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'



    # Group key includes LOS_Bin
    group_key = (pdx,pdx_chg,apdx_status,bpdx_status)

    # Update stats
    stats[group_key]['Total_Claims'] += 1

    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result > 0:
            stats[group_key]['Total_Savings'] += float(audit_result)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    pdx,pdx_chg,apdx_status,bpdx_status = key
    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)

    denial_percent = round(denied / total_claims * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0

    final_data.append({
       # 'DRG': drg,
        'PRIM_DX': pdx,
        'P_DX_What_Chg': pdx_chg,
        'A_PDX_Status': apdx_status,
        'B_PDX_Status': bpdx_status,

        'Total_Claims': total_claims,
        'Approved': approved,
        'Denied': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_Per_Claim': avg_saving_per_claim
    })

# -----------------------------
# 8. Export to Excel
# -----------------------------
output_df = pd.DataFrame(final_data)
output_df = output_df.sort_values(
    by=['Total_Claims'],
    ascending=[False]
)

# output_file = "DRG_PDX_SDXSet_LOSBin_Simplified_Analysis.xlsx"
# output_df.to_excel(output_file, index=False)

#print(f"\n✅ Simplified LOS-Bin analysis completed for {len(output_df)} groups.")
# print(f"📊 Results saved to '{output_file}'")

In [ ]:
output_df.head(10)

In [ ]:
df_PDX_resequence_a419 = df_PDX_resequence[df_PDX_resequence['PRIM_DX']=='A419']
df_PDX_resequence_a419.shape

In [ ]:
from collections import defaultdict

# -----------------------------
# 5. Initialize stats dictionary
# -----------------------------
# Key: (DRG, PRIM_DX, SDX_Set, LOS_Bin)
stats = defaultdict(lambda: {
    'Total_Claims': 0,
    'Approved': 0,
    'Denied': 0,
    'Total_Savings': 0.0
})

# -----------------------------
# 6. Loop through df_apr
# -----------------------------
for idx, row in df_PDX_resequence_a419.iterrows():
    # Skip if LOS_Bin is missing
    if pd.isna(row['P_DX_What_Change_Status']):
        continue
        
    drg = row['ADRG']
    pdx = str(row['PRIM_DX']).strip().upper()
    pdx_chg = row['P_DX_What_Change_Status']
    pdx_status = row['PDX_Presence_Tag']
    audit_result = row['IDSavings']
   # chap_set = row['CHAPTER_SET']
    status = 'APPROVED' if audit_result == 0 else 'DENIED'



    # Group key includes LOS_Bin
    group_key = (pdx,pdx_chg,pdx_status)

    # Update stats
    stats[group_key]['Total_Claims'] += 1

    if status == 'APPROVED':
        stats[group_key]['Approved'] += 1
    else:
        stats[group_key]['Denied'] += 1
        if audit_result > 0:
            stats[group_key]['Total_Savings'] += float(audit_result)

# -----------------------------
# 7. Build Final DataFrame
# -----------------------------
final_data = []

for key, data in stats.items():
    pdx,pdx_chg,pdx_status = key
    total_claims = data['Total_Claims']
    approved = data['Approved']
    denied = data['Denied']
    total_savings = round(data['Total_Savings'], 2)

    denial_percent = round(denied / total_claims * 100, 2) if total_claims else 0
    avg_saving_per_claim = round(total_savings / total_claims, 2) if total_claims else 0

    final_data.append({
       # 'DRG': drg,
        'PRIM_DX': pdx,
        'P_DX_What_Chg': pdx_chg,
        'PDX_Status': pdx_status,
        'Total_Claims': total_claims,
        'Approved': approved,
        'Denied': denied,
        'Denial_Percent': denial_percent,
        'Total_Savings': total_savings,
        'Avg_Saving_Per_Claim': avg_saving_per_claim
    })

# -----------------------------
# 8. Export to Excel
# -----------------------------
output_df = pd.DataFrame(final_data)
output_df = output_df.sort_values(
    by=['Denied'],
    ascending=[False]
)

output_df.head()

# output_file = "DRG_PDX_SDXSet_LOSBin_Simplified_Analysis.xlsx"
# output_df.to_excel(output_file, index=False)

#print(f"\n✅ Simplified LOS-Bin analysis completed for {len(output_df)} groups.")
# print(f"📊 Results saved to '{output_file}'")

In [ ]:
df_PDX_resequence_a419['PDX_Presence_Tag'].value_counts()

In [ ]:
output_df.to_excel("Resquence_PDX_A419.xlsx")

In [ ]:
df_PDX_resequence_a419['P_DX_What_Change_Status'].value_counts()

In [ ]:
df2['PDX_Presence_Tag'].value_counts()

In [ ]:
df_PDX_same = df2[df2['PDX_Presence_Tag']=='Present in B_PDX']
df_PDX_same.shape

In [ ]:
df_PDX_removed = df2[df2['PDX_Presence_Tag']=='A_PDX_Removed']
df_PDX_removed.shape

In [ ]:
df_PDX_removed['InitialDeterminationStatus'].value_counts()

In [ ]:
df_PDX_sdx = df2[df2['PDX_Presence_Tag']=='Present in SDX']
df_PDX_sdx.shape

In [ ]:
df_PDX_sdx['InitialDeterminationStatus'].value_counts()

In [ ]:
df_PDX_sdx['InitialDeterminationStatus'].value_counts()

In [ ]:
df2["P_DX_Flag"] = df2.apply(
    lambda x: "ReSequence" 
    if x["B_PRIMDX_Mapped"] in (x["A_DX_Type_list"] if isinstance(x["A_DX_Type_list"], list) else [])
    else "Same",
    axis=1
)

df2["A_Prim_Tag"] = df2.apply(
    lambda x: (
        "Replace to SDX"
        if x["PRIM_DX"] in (x["B_DX_Type_list"] if isinstance(x["B_DX_Type_list"], list) else [])
        else "Same"
        if x["P_DX_Flag"] == "Same"
        else "Removed"
    ),
    axis=1
)


In [ ]:
def get_removed(a_list, b_list):
    if not isinstance(a_list, list) or not isinstance(b_list, list):
        return []
    return [x for x in a_list if x not in b_list]


def detect_removed_tag(row):
    removed_sdx = get_removed(row["A_DX_Set"], row["B_DX_Set"])


    if len(removed_sdx) > 0:
        return "Removed", removed_sdx, f"{len(removed_sdx)} removed out of {len(row['A_DX_Set'])}"
    else:
        return "None", [], "Nothing Removed"

# Apply
df2[["SDX_Tag_Removed", "Removed_List", "Removed_Count"]] = (
    df2.apply(lambda row: pd.Series(detect_removed_tag(row)), axis=1)
)

In [ ]:
columns_to_keep = [
    "Payor","claimid","Client","Control_ID","Claim_Number","PaymentType","Group_Type","Payments",
    "Tax_ID","Provider_ID","HospitalName","LOS","LOS_Bin","DischargeYear","DischargeMonth","AGE","AGE_bin",
    "LOB","MRREQ","MRREC","DISP_Code","ADRG","PRIM_DX","B_DenialCode","B_DRG","B_PRIMDX","B_PRIMDX_Clean","B_PRIMDX_Mapped",
    "A_DX_List","B_DX_List","A_DX_List_Clean",
    "B_DX_List_Clean","A_DX_Type_list","B_DX_Type_list","A_DX_Type", "A_DX_Set","B_DX_Type", "B_DX_Set",
    "InitialDeterminationDate","InitialDenialType",
    "InitialDeterminationStatus","IDSavings","AdjustedSavings","P_DX_Flag","A_Prim_Tag","SDX_Tag_Removed", "Removed_List", "Removed_Count"
]

df_clean = df2[columns_to_keep].copy()
df_clean.head(1)

In [ ]:
matched_df = df_clean[
    df_clean["PRIM_DX"].str.strip().str.upper() ==
    df_clean["B_PRIMDX_Clean"].str.strip().str.upper()
]
matched_df.info()


In [ ]:
matched_df['InitialDeterminationStatus'].value_counts()

In [ ]:
matched_df1 = df_clean[~(
    df_clean["PRIM_DX"].str.strip().str.upper() ==
    df_clean["B_PRIMDX_Clean"].str.strip().str.upper())
]
matched_df1.info()


In [ ]:
matched_df1.head()

In [ ]:
filtered_df = df_clean[
    (df_clean["P_DX_Flag"] == "Same") &
    (df_clean["A_Prim_Tag"] == "Same") &
    (df_clean["SDX_Tag_Removed"] == "None") &
    (df_clean["Removed_List"].str.len() == 0) &
    (df_clean["Removed_Count"] == "Nothing Removed")
]
filtered_df.info()

In [ ]:
filtered_df['InitialDeterminationStatus'].value_counts()

In [ ]:
filtered_df1 = df_clean[~(
    (df_clean["P_DX_Flag"] == "Same") &
    (df_clean["A_Prim_Tag"] == "Same") &
    (df_clean["SDX_Tag_Removed"] == "None") &
    (df_clean["Removed_List"].str.len() == 0) &
    (df_clean["Removed_Count"] == "Nothing Removed"))
]
filtered_df1.info()

In [ ]:
filtered_df1.head()

In [ ]:
filtered_df2 = filtered_df1[(
    (filtered_df1["P_DX_Flag"] == "Same") &
    (filtered_df1["A_Prim_Tag"] == "Same")
)
]
filtered_df2.info()